# Helpers

In [1]:
import sys
sys.path.append("/anaconda/lib/python2.7/site-packages")
import pandas as pd
import shapefile 
import json
import os
from sklearn.neighbors import KNeighborsClassifier as KNN

### Code to get neighborhoods from lat/long

In [2]:
#################################################################################################
####### Inputs:
#######     - path to shapefile
#######		- name of the attribute we want as our name of each region
####### Returns:
#######		- dict mapping region names to shape polygons 

def generate_regions_to_points_dict(shapefile_path, region_attribute_name):
    sf = shapefile.Reader(shapefile_path)
    shapes = sf.shapes()
    records = sf.records()
    fields = sf.fields
    
    # Make sure region_attribute_name exists in the shapefile
    found = False
    for i in range(1, len(fields)):
        if fields[i][0] == region_attribute_name:
            attribute_key = i-1
            found = True
    if not found:
        print "Region attribute not found. Available attributes:"
        for i in range(1, len(fields)):
            print str(fields[i][0]) + "\t",
        return     

    
    regions_to_points_dict = {}
    for i in range(len(records)):
        regions_to_points_dict[records[i][attribute_key]] = shapes[i].points
    return regions_to_points_dict
 



#################################################################################################
####### Code from: http://www.ariel.com.au/a/python-point-int-poly.html
####### Inputs:
####### 	- x value
#######		- y value
#######		- polygon: list of (x,y) pairs corresponding to the perimeter/corners of a polygon
####### Returns: true if x,y is in the specified polygon

def point_inside_polygon(x,y,poly):

    n = len(poly)
    inside =False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xinters = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside




#################################################################################################
####### Inputs:
#######     - latitude
#######		- longitude 
#######     - regions_to_points_dict - keys are region names, values are lists of points outlining the region
#######								- see "generate_regions_to_points_dict" method to create one of these dicts
####### Returns:
#######		- region the latitude,longitude input falls inside 

def get_region_from_latlong_and_shapefile(latitude, longitude, regions_to_points_dict, print_errors=True):
    for key in regions_to_points_dict.keys():
        if point_inside_polygon(longitude, latitude, regions_to_points_dict[key]):
            return key
    if print_errors:
        print "region not found"
    return


### Read in data and format columns

In [3]:
# generate mapping for lat/long to neighborhoods
path_to_shape_file = '/Users/nbw/Desktop/cs171_test_files/ZillowNeighborhoods-NY/ZillowNeighborhoods-NY'
neighborhood_dict = generate_regions_to_points_dict(path_to_shape_file, 'NAME')
    
# generate mapping from neighborhoods to boroughs
sf = shapefile.Reader(path_to_shape_file)
records = sf.records()

boroughs_list =  ["New York City-Manhattan", "New York City-Brooklyn", "New York City-Queens", "New York City-Bronx", "New York City-Staten Island"]

boroughs_list =  ["New York City-Manhattan", "New York City-Brooklyn", "New York City-Queens", "New York City-Bronx", "New York City-Staten Island"]
neighborhood_to_borough_dict = {}
for record in records:
    if record[2] in boroughs_list:
        neighborhood_to_borough_dict[record[3]] = record[2]

In [6]:
path_to_datafiles = "/Users/nbw/Dropbox/CS_171_data/Airbnb Files/"

In [6]:
counter = 0
for filename in os.listdir(path_to_datafiles):
    counter += 1
    print counter 
    
    # make sure file is right format
    if "NYC_listings" not in filename or "ts.csv" in filename or filename=="2015-01-01_NYC_listings.csv":
        continue

    

    

1
2
3
4
5
6
7


//anaconda/envs/tensorflow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (26,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyboardInterrupt: 

In [14]:

filename = '2016-10-01_NYC_listings.csv'
df = pd.read_csv(path_to_datafiles + filename)



# rename columns to match standardized names
df.rename(columns={'id': 'room_id', 'number_of_reviews': 'num_reviews', \
        'minimum_nights': 'min_stay', 'last_scraped': 'collected'}, inplace=True)

# convert iabb's price from "$__" strings to floats
df['price'] = df['price'].apply(lambda x: float(x.split("$")[1].replace(",", "")))

# calculate ratings from composite values
df["rating"] = None
for i in df.index.values:
    if df['review_scores_accuracy'].iloc[i] == df['review_scores_accuracy'].iloc[i]:
        scores_sum = df['review_scores_accuracy'].iloc[i] + df['review_scores_cleanliness'].iloc[i] + \
        df['review_scores_checkin'].iloc[i] + df['review_scores_communication'].iloc[i] + \
        df['review_scores_location'].iloc[i] + df['review_scores_value'].iloc[i]  
        df.set_value(i, "rating", scores_sum/12.0)

# drop unwanted columns
cols_to_keep = [u'room_id', u'collected', u'host_id', u'latitude', u'longitude', u'room_type', u'accommodates', \
u'bathrooms', u'bedrooms', u'price', u'min_stay', u'num_reviews', u'calculated_host_listings_count', u'rating', u'source', \
u'illegal', u'taxes', u'date', u'property_type', u'host_location', u'reviews_per_month', u'maximum_nights']
df = df.drop(list(set(df.columns) - set(cols_to_keep)), axis=1)



df["neighborhood"] = None
for i in df.index.values:
    df.set_value(i, 'neighborhood', get_region_from_latlong_and_shapefile(df['latitude'].iloc[i] , df['longitude'].iloc[i] , neighborhood_dict, print_errors=False))

# fill missing values using sklearn's knn functions
unknown = df[df["neighborhood"] != df["neighborhood"]]
known = df[df["neighborhood"] == df["neighborhood"]]

train_x_matrix = known[["latitude", "longitude"]].values
train_y_matrix = known["neighborhood"]
test_x_matrix = unknown[["latitude", "longitude"]].values

model = KNN(n_neighbors = 1)
model.fit(train_x_matrix, train_y_matrix)
preds = model.predict(test_x_matrix)

unknown["neighborhood"] = preds

df.loc[unknown.index, 'neighborhood'] = pd.Series(unknown["neighborhood"])

# get boroughs from neighborhood
df["borough"] = None
for i in df.index.values:
    if df['neighborhood'].iloc[i]:
        df.set_value(i, 'borough', neighborhood_to_borough_dict[df['neighborhood'].iloc[i]])

df['borough'] = df['borough'].apply(lambda x: x.split("-")[1] if x else None)

# add identifier columns
df["date"] = filename.split("_")[0]
df["source"] = "iabb"


/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
# CALCULATE TAXES

df["tax_total"] = 0
df["tax_NYC_sales"] = 0
df["tax_state_sales"] = 0
df["tax_mctd"] = 0
df["tax_hotel_occupancy"] = 0
df["tax_javits"] = 0

df["illegal"] = 0

avgLength = 6.4;
maxNights = 30 * .7;

for i in df.index.values:

    stayLength = max(df['min_stay'].iloc[i], avgLength)
    revs = df['reviews_per_month'].iloc[i]
    if revs==revs:
        nights_booked = min(revs * 2 * stayLength, maxNights)
    else:
        nights_booked = 0


    month_revenue = nights_booked * df['price'].iloc[i];

    hotel_tax_applies = (df['maximum_nights'].iloc[i] > 14)

    bedrooms = df['bedrooms'].iloc[i]
    if df['room_type'].iloc[i] == "Entire home/apt" and bedrooms==bedrooms:
        num_rooms = bedrooms + 1
    elif bedrooms==bedrooms:
        num_rooms = bedrooms
    else:
        num_rooms = 1

    tax_NYC_sales = .045 * month_revenue;
    tax_state_sales = .04 * month_revenue;
    tax_mctd = .00375 * month_revenue;
    if hotel_tax_applies: 
        tax_hotel_occupancy = .05875 * month_revenue + 2 * num_rooms * nights_booked
        tax_javits = 1.5 * num_rooms * nights_booked
    else:
        tax_hotel_occupancy = 0
        tax_javits = 0
    tax_total = tax_NYC_sales + tax_state_sales + tax_mctd + tax_hotel_occupancy + tax_javits
    
    df.set_value(i, "tax_total", tax_total)
    df.set_value(i, "tax_NYC_sales", tax_NYC_sales)
    df.set_value(i, "tax_state_sales", tax_state_sales)
    df.set_value(i, "tax_mctd", tax_mctd)
    df.set_value(i, "tax_hotel_occupancy", tax_hotel_occupancy)
    df.set_value(i, "tax_javits", tax_javits)
    
    host_loc = df['host_location'].iloc[i]
    if host_loc != host_loc: 
        if (df['property_type'].iloc[i] == "Apartment") and (df['min_stay'].iloc[i] < 30) and (df['room_type'].iloc[i] == "Entire home/apt" or df["calculated_host_listings_count"].iloc[i] > 1):
            df.set_value(i, "illegal", 1)
    else:
        if (df['property_type'].iloc[i] == "Apartment") and (df['min_stay'].iloc[i] < 30) and (df['room_type'].iloc[i] == "Entire home/apt" or df["calculated_host_listings_count"].iloc[i] > 1 or (not "New York" in host_loc and not "NY" in host_loc and host_loc != "US")):
            df.set_value(i, "illegal", 1)
        


In [18]:
df = df.drop([u'maximum_nights', u'reviews_per_month'], axis=1)

In [19]:
print "Percent illegal: " , sum(df["illegal"])/float(len(df))
print "Taxes for this month:"
print "NYC_sales: " , sum(df["tax_NYC_sales"]) 
print "state_sales: " , sum(df["tax_state_sales"]) 
print "mctd: " , sum(df["tax_mctd"]) 
print "hotel_occupancy: " , sum(df["tax_hotel_occupancy"]) 
print "javits: " , sum(df["tax_javits"]) 
print "total:" , sum(df["tax_total"]) 

Percent illegal:  0.615022880692
Taxes for this month:
NYC_sales:  2463799
state_sales:  2189272
mctd:  190660
hotel_occupancy:  4044527
javits:  858214
total: 9801908


In [20]:
df

,room_id,collected,host_id,host_location,latitude,longitude,property_type,room_type,accommodates,bathrooms,...,borough,date,source,tax_total,tax_NYC_sales,tax_state_sales,tax_mctd,tax_hotel_occupancy,tax_javits,illegal
0,4717459,2016-10-02,20540837,"New York, New York, United States",40.760433,-73.995519,Apartment,Entire home/apt,4,1.0,...,Manhattan,2016-10-01,iabb,42,10,9,0,18,3,1
1,9436049,2016-10-02,7117596,"New York, New York, United States",40.689637,-73.948845,Apartment,Private room,2,1.0,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,0
2,6028165,2016-10-02,31287099,"New York, New York, United States",40.716924,-73.990924,Apartment,Entire home/apt,2,1.0,...,Manhattan,2016-10-01,iabb,102,25,22,2,44,8,1
3,5604005,2016-10-02,3852579,US,40.720923,-73.993861,Apartment,Entire home/apt,4,2.0,...,Manhattan,2016-10-01,iabb,1149,283,252,23,496,94,1
4,11271036,2016-10-02,8145820,"Marburg, Hesse, Germany",40.796060,-73.948810,Apartment,Private room,1,1.5,...,Manhattan,2016-10-01,iabb,14,2,2,0,6,2,1
5,3254762,2016-10-02,16460800,"New York, New York, United States",40.718557,-73.941999,Apartment,Private room,1,1.0,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,0
6,7469908,2016-10-02,31788441,"New York, New York, United States",40.769715,-73.955023,Apartment,Entire home/apt,2,1.0,...,Manhattan,2016-10-01,iabb,569,151,134,12,239,31,1
7,12581749,2016-10-02,68235110,US,40.677253,-73.969025,Loft,Private room,16,0.5,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,0
8,523123,2016-10-02,2572253,Brooklyn NY,40.660509,-73.984945,Apartment,Entire home/apt,6,2.0,...,Brooklyn,2016-10-01,iabb,80,41,36,3,0,0,1
9,4848500,2016-10-02,6902571,"New York, New York, United States",40.688858,-73.952119,Apartment,Entire home/apt,3,1.0,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,1


In [13]:
df

,room_id,collected,host_id,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,borough,date,source,tax_total,tax_NYC_sales,tax_state_sales,tax_mctd,tax_hotel_occupancy,tax_javits,illegal
0,4717459,2016-10-02,20540837,40.760433,-73.995519,Apartment,Entire home/apt,4,1.0,1.0,...,Manhattan,2016-10-01,iabb,42,10,9,0,18,3,1
1,9436049,2016-10-02,7117596,40.689637,-73.948845,Apartment,Private room,2,1.0,1.0,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,0
2,6028165,2016-10-02,31287099,40.716924,-73.990924,Apartment,Entire home/apt,2,1.0,1.0,...,Manhattan,2016-10-01,iabb,102,25,22,2,44,8,1
3,5604005,2016-10-02,3852579,40.720923,-73.993861,Apartment,Entire home/apt,4,2.0,2.0,...,Manhattan,2016-10-01,iabb,1149,283,252,23,496,94,1
4,11271036,2016-10-02,8145820,40.796060,-73.948810,Apartment,Private room,1,1.5,1.0,...,Manhattan,2016-10-01,iabb,14,2,2,0,6,2,1
5,3254762,2016-10-02,16460800,40.718557,-73.941999,Apartment,Private room,1,1.0,1.0,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,0
6,7469908,2016-10-02,31788441,40.769715,-73.955023,Apartment,Entire home/apt,2,1.0,0.0,...,Manhattan,2016-10-01,iabb,569,151,134,12,239,31,1
7,12581749,2016-10-02,68235110,40.677253,-73.969025,Loft,Private room,16,0.5,1.0,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,0
8,523123,2016-10-02,2572253,40.660509,-73.984945,Apartment,Entire home/apt,6,2.0,4.0,...,Brooklyn,2016-10-01,iabb,80,41,36,3,0,0,1
9,4848500,2016-10-02,6902571,40.688858,-73.952119,Apartment,Entire home/apt,3,1.0,2.0,...,Brooklyn,2016-10-01,iabb,0,0,0,0,0,0,1


In [21]:
# convert df to json array and write to file
json_array = df.to_json(orient='records')

with open(filename.split("_")[0] + "_with_analyses.json", 'w') as f:
#with open("jsondata/"+filename.split("_")[0] + ".json", 'w') as f:
    f.write(json_array)
print filename.split("_")[0] + ".json written"

2016-10-01.json written
